In [1]:
!pip install unzip
#!pip install -r requirements.txt
!pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for unzip: filename=unzip-1.0.0-py3-none-any.whl size=1281 sha256=86d41bc8216f4ceeb48390834f72045edb4146aa997ba421cd1ae91b9d743a3e
  Stored in directory: /root/.cache/pip/wheels/fb/5b/81/0f3e1e533b52883f88ab978178c15627a4fce4c13f74911dce
Successfully built unzip
  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=39c2e644c2a837f80d9b71ba152356db521da68e4523eda92bb104863fb51cf0
  Stored in directory: /root/.cache/pip/wheels/b6/a1/81/1036477786ae0e17b522f6f5a838f9bc4288d1016fc5d0e1ec
Successfully built utils


# Introducción

El objetivo principal de los algoritmos de _matching_ es el de, dado un fragmento de texto, encontrar, de entre un conjunto de candidatos, los textos más similares al fragmento orginial.

Como texto podemos pensar tanto en palabras, en pequeñas frases o en documentos enteros.

Podemos pensar en 3 tipos de técnicas de matching:
- **Coincidencia exacta**: ya vimos ejemplos de este tipo al estudiar la **Distancia de Edición**.
    - A nivel de carácter: strings que difieren en caracteres
    - A nivel de token: strings que difieren en palabras
    - Fonéticos: palabras que suenan de manera similar
- **Coincidencia aproximada o difusa**
- **Coincidencia mediante aproximaciones**

| Candidato / Tipo de resultado 	| Exacta 	| Aproximada 	| Transformación 	|
|-	|-	|-	|-	|
| String 	| Comparación de strings 	| Comparación difusa 	| Ontologías 	|
| Categoría 	| Gramáticas 	| Reconocimiento probabilístico 	| Análisis de topics 	|
| Documento 	| - 	| Recuperación de información 	| Traducción automática 	|

# Regular expressions (Regex)

Muy utilizadas (y conocidas) suelen emplearse al limpiar el texto o buscar formatos dentro del texto. A modo introductorio, las expresiones regulares son una forma de finite state automaton.

<img src=http://www.cs.cornell.edu/courses/cs312/2006fa/recitations/images/dfa-examples.gif>

Son grafos que siguen una secuencia que nosotros definimos. Por ejemplo, el grafo de la izquierda, solo podría generar expresiones como ab, abb, abbb, abbbb y así hasta el infinito. El de la derecha, podría generar expresiones como abcb, o abbb, abbbbbb, por ejemplo.

Conceptualmente, las regex _funcionan_ así _por debajo_. Lógicamente cuando las usamos es mucho más fácil, ¿verdad :D?

La definición de estos grafos es posible mediante la [librería de Python re](https://docs.python.org/3/library/re.html), módulo del paquete base de Python dedicado a las expresiones regulares.

Cierto es que no siempre nos hará falta. Algunas veces con un simple _string.replace()_ o _string.find()_ tendremos suficiente. No obstante, para muchas tareas son bastante útiles.

Algunas tareas típicas en las que se utilizan son la búsqueda (y a veces normalización) de emails, urls, numeros de telefono, etc. Solo la extracción es interesante, pero mediante su normalización nos permite reducir la cardinalidad del vocabulario y asociar entidades similares a un mismo alias.

[Regex Online](https://regexr.com/) es uno de los mejores recursos online para visualizar que hacen los regex

Veamos algunos ejemplos.

In [2]:
# Función que nos ayudará a visualizar algunos resultados

from termcolor import colored
def test_pass(ok, text):
    color = 'green' if ok else 'red'
    return colored(text, color)

In [3]:
import re

In [4]:
text = 'Todos los animales son iguales, pero algunos son más iguales que otros'

In [5]:
RE_TEST = re.compile(r'todos')
print(RE_TEST.match(text))

None


In [6]:
RE_TEST = re.compile(r'Todos')
print(RE_TEST.match(text))

<re.Match object; span=(0, 5), match='Todos'>


In [7]:
RE_TEST = re.compile(r'[a-zA-Z]')
print(RE_TEST.match(text))

<re.Match object; span=(0, 1), match='T'>


In [8]:
RE_TEST = re.compile(r'\bTodos\b')
print(RE_TEST.match(text))

<re.Match object; span=(0, 5), match='Todos'>


In [9]:
RE_TEST = re.compile(r'\bTod\b')
print(RE_TEST.match(text))

None


## Obtener un correo electrónico

In [10]:
"""
^ -> start of string
+ -> match 1 or more preceding regex
[^@]+
@[^@]+
\. -> '.'
"""

RE_EMAIL = re.compile('[^@]+@[^@]+\.[^@]+')

<>:6: SyntaxWarning: invalid escape sequence '\.'
<>:9: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\.'
<>:9: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-3680466631.py:6: SyntaxWarning: invalid escape sequence '\.'
  \. -> '.'
/tmp/ipython-input-3680466631.py:9: SyntaxWarning: invalid escape sequence '\.'
  RE_EMAIL = re.compile('[^@]+@[^@]+\.[^@]+')


In [11]:
emails_list = [
    '@invalid@adress.com',
    'correo_valido@gmail.com',
    'notan@valido@gmail.com',
    'si.valido.david@gmail.com',
    'paginaweb.com',
    'paginaweb.com@paginaweb.com'
]
for email in emails_list:
    if RE_EMAIL.match(email):
        print(True)
        print(test_pass(True, email))
        print('___')
    else:
        print(False)
        print(test_pass(False, email))
        print('___')

False
@invalid@adress.com
___
True
correo_valido@gmail.com
___
False
notan@valido@gmail.com
___
True
si.valido.david@gmail.com
___
False
paginaweb.com
___
True
paginaweb.com@paginaweb.com
___


## Obtener precios

In [12]:
from random import shuffle
import unicodedata

CURRENCIES = ''.join(chr(i) for i in range(0xffff) if unicodedata.category(chr(i)) == 'Sc')
RE_MONEY_GENERAL= re.compile('((\s|^)([\d]*)(\.)?([\d])*([%s]|e|USD|USD\$|U\$D)(\s|$))'
                          '|((\s|^)([%s]|e|USD|USD\$|U\$D)([\d])*(\.)?([\d])*(\s|$))'%(CURRENCIES, CURRENCIES), re.IGNORECASE)
RE_MONEY_EU= re.compile('((\s|^)([\d]{0,3}([\.][\d]{3})(,[\d]*))([%s]|e|(USD|USD\$|U\$D))(\s|$))'
                     '|((\s|^)([%s]|e|(USD|USD\$|U\$D))([\d]{0,3}([\.][\d]{3})(,[\d]*))(\s|$))'%(CURRENCIES, CURRENCIES), re.IGNORECASE)
RE_MONEY_EU_INVERSE= re.compile('((\s|^)([\d]{0,3}([,][\d]{3})(\.[\d]*))([%s]|e|(USD|USD\$|U\$D))(\s|$))'
                             '|((\s|^)([%s]|e|(USD|USD\$|U\$D))([\d]{0,3}([,][\d]{3})(\.[\d]*))(\s|$))'%(CURRENCIES, CURRENCIES), re.IGNORECASE)


<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-2203267955.py:5: SyntaxWarning: invalid escape sequence '\s'
  RE_MONEY_GENERAL= re.compile('((\s|^)([\d]*)(\.)?([\d])*([%s]|e|USD|USD\$|U\$D)(\s|$))'
/tmp/ipython-input-2203267955.py:6: SyntaxWarning: invalid escape sequence '\s'
  '|((\s|^)([%s]|e|USD|USD\$|U\$D)([\d])*(\.)?([\d])*(\s|$))'%(CURRENCIES, CURRENCIES), re.IGNORECASE)
/tmp/ipython-input-2203267955.py:7: SyntaxWar

In [13]:
correct_currencies = [
    '$20.2',
    '$.2',
    '$0.2',
    '$3433.2',
    '.2$',
    '2.0$',
    '2.$',
    '2.0€',
    '2¥',
    '20USD',
    '20e',
    '20 €',
    '20 usd',
    '€200.123,2',
    '2.134,56$',
    '23232₽',
    '334,222.20€',
    '20U$D',
    '$200']

incorrect_currencies = [
    'asdfsd',
    '$asdasd',
    '23333,444.20€',
    '€34523sdfas',
    '€213.sd',
    '$3vg554.25',
    'expensive',
    'cheap',
    '2342,222.90€'
]

all_currencies = correct_currencies + incorrect_currencies
shuffle(all_currencies)

for currency in all_currencies:
    if RE_MONEY_GENERAL.match(currency) or RE_MONEY_EU.match(currency) or RE_MONEY_EU_INVERSE.match(currency):
        print(test_pass(True, currency))
    else:
        print(test_pass(False, currency))

$20.2
€213.sd
.2$
2.134,56$
cheap
2¥
$asdasd
2342,222.90€
2.0$
expensive
$.2
20 €
334,222.20€
2.0€
$200
€200.123,2
2.$
23232₽
asdfsd
20e
€34523sdfas
$3vg554.25
$3433.2
20USD
23333,444.20€
$0.2
20U$D
20 usd


# DAWG

Lo presentábamos antes de manera muy  rápida, un _Directed Acyclic Word Graph_ (por sus siglas, DAWG), también llamado, _Deterministic Acyclic Finite State Automaton_ (DAFSA), es un tipo de estructura de datos que permite representar datos de tipo texto y realizar consultas.

![image.png](attachment:image.png)

En el grafo generado se distinguen:
- **Nodos**: un carácter / símbolo
- **Vértices**: enlace con el siguiente carácter / símbolo más probable

http://www.wutka.com/dawg.html


## Ejemplos

In [14]:

from utils import load_movie_titles

In [16]:
datasets_path = './'
movie_titles_file = 'films.txt'

In [17]:
movies_titles = load_movie_titles(datasets_path, movie_titles_file)

## Lo creamos

In [18]:
!pip install shell

In [19]:
!pip install pydawg

  Preparing metadata (setup.py) ... done
  Created wheel for pydawg: filename=pyDAWG-1.0.1-cp312-cp312-linux_x86_64.whl size=63710 sha256=e827fb4c540edc24c13f9849b887131fd59b66a8f4fc96ef3e3562a9b4e3a4ea
  Stored in directory: /root/.cache/pip/wheels/57/f0/6b/678352fda0b53b13ba0a40e91642a402c3899b0c1aea6ed07e
Successfully built pydawg


In [20]:
!pip install marisa-trie
import marisa_trie



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.6 MB/s eta 0:00:00


In [21]:
# Crear lista de palabras (titulos de películas)
words = sorted(m.title for m in movies_titles)

# Crear el trie
trie = marisa_trie.Trie(words)

# Ejemplo de uso:
print("Inception" in trie)  # True/False
print(trie.keys("The"))     # Devuelve todos los títulos que empiezan con "The"

False
['The Black Cat', 'The Black Castle', 'The Black Hole', 'The Black Sleep', "The Blackcoat's Daughter", 'The Blade', 'The Blair Witch Project', 'The Blob', "The Blood of Dracula's Castle", 'The Blue Max', 'The Blues Brothers', 'The Bling Ring', 'The Bride of Frankenstein', 'The Brides of Dracula', 'The Bridge on the River Kwai', 'The Brick Dollhouse', "The Brink's Job", 'The Brotherhood', 'The Brotherhood of Satan', 'The Brothers Rico', 'The Brothers Warner', 'The Brood', 'The Breakfast Club', 'The Breed', "The Brain That Wouldn't Die", 'The Bad News Bears', 'The Bad News Bears go to Japan', 'The Bad News Bears in Breaking Training', 'The Bad and the Beautiful', 'The Bat', 'The Battle at Elderbush Gulch', 'The Band Wagon', 'The Bandit of Sherwood Forest', 'The Baby', 'The Ballad of Nessie', 'The Barrens', 'The Big Bus', 'The Big Country', 'The Big Lebowski', 'The Big Parade', 'The Big Red One', 'The Big Trail', 'The Bird With the Crystal Plumage', 'The Birds', 'The Bible: In the B

In [22]:
import random
# Crear lista de palabras (títulos de películas)
words = [m.title for m in movies_titles]

# Crear el Trie
trie = marisa_trie.Trie(words)

# Elegir un título al azar
t = random.choice(movies_titles).title
print("Título elegido:", t)



Título elegido: The Night Strangler


In [23]:
# Verificar si está en el Trie
print(t in trie)  # True si el título existe

True


## Operaciones

### Búsqueda por prefijo

In [24]:
# Crear lista de palabras (títulos de películas)
words = [m.title for m in movies_titles]

# Crear el Trie
trie = marisa_trie.Trie(words)

# Buscar todos los títulos que contienen "Batman" como prefijo
for title in trie.keys("Batman"):
    print(title)

Batman
Batman Returns
Batman: The Movie


### Prefijo más largo

trie.prefixes(s) devuelve todos los prefijos de s que están en el Trie.

max(prefixes, key=len) selecciona el más largo.

Si no hay coincidencia, se maneja con un if para evitar errores.

In [25]:
# Crear lista de palabras (títulos de películas)
words = [m.title for m in movies_titles]

# Crear el Trie
trie = marisa_trie.Trie(words)

# Texto de ejemplo
s = 'La guerra de nunca jamás'

# Obtener todos los prefijos existentes en el Trie
prefixes = trie.prefixes(s)

# Escoger el prefijo más largo
if prefixes:
    longest = max(prefixes, key=len)
    print("Prefijo más largo:", longest)
else:
    print("No hay prefijo coincidente")

No hay prefijo coincidente


### Búsqueda en una oración

In [26]:


# Crear lista de palabras (títulos de películas)
words = [m.title for m in movies_titles]
trie = marisa_trie.Trie(words)

# Función que busca la coincidencia más larga desde el inicio de la lista de tokens
def token_match(trie, tknlist):
    for n in range(len(tknlist), 0, -1):
        test_str = ' '.join(tknlist[:n])
        if test_str in trie:  # Se usa "in trie" en lugar de "in dawg"
            return test_str
    return None  # Si no encuentra coincidencia

# Función que aplica token_match a todos los chunks del utterance
def token_match_all(trie, utterance):
    tknlist = utterance.split()
    return [token_match(trie, tknlist[chunk:])
            for chunk in range(len(tknlist))]


In [27]:
utterance = "La guerra de nunca jamás es épica"
matches = token_match_all(trie, utterance)
print(matches)


[None, None, None, None, None, None, None]


In [28]:


# Lista de títulos de ejemplo
movies_titles = [
    type('Movie', (), {'title': 'La guerra de nunca jamás'}),
    type('Movie', (), {'title': 'Batman Begins'}),
    type('Movie', (), {'title': 'Superman Returns'})
]

# Crear lista de palabras
words = [m.title for m in movies_titles]

# Crear el Trie
trie = marisa_trie.Trie(words)

# Función token_match
def token_match(trie, tknlist):
    for n in range(len(tknlist), 0, -1):
        test_str = ' '.join(tknlist[:n])
        if test_str in trie:
            return test_str
    return None

# Función token_match_all
def token_match_all(trie, utterance):
    tknlist = utterance.split()
    return [token_match(trie, tknlist[chunk:])
            for chunk in range(len(tknlist))]

# Frase que coincide completamente con títulos en el Trie
utterance = "La guerra de nunca jamás Batman Begins"

matches = token_match_all(trie, utterance)
print("Frase:", utterance)
print("Coincidencias:", matches)


Frase: La guerra de nunca jamás Batman Begins
Coincidencias: ['La guerra de nunca jamás', None, None, None, None, 'Batman Begins', None]


Devuelve una lista con las coincidencias más largas que encuentra en Trie para cada chunk de Tokens


### Minimal perfect hash

In [29]:



# Lista de títulos de ejemplo
movies_titles = [
    type('Movie', (), {'title': 'La guerra de nunca jamás'}),
    type('Movie', (), {'title': 'Batman'}),
    type('Movie', (), {'title': 'Superman'})
]

words = [m.title for m in movies_titles]

# Asignar un índice a cada palabra
values = [(i,) for i in range(len(words))]  # Cada valor debe ser tupla

# Crear RecordTrie con un formato de entero ('I' = unsigned int)
trie = marisa_trie.RecordTrie('I', zip(words, values))

# Obtener el índice de una palabra
word = 'Batman'
if word in trie:
    index = trie[word][0][0]  # trie[word] devuelve lista de tuplas
    print(f"El índice de '{word}' es {index}")
else:
    print(f"'{word}' no está en el Trie")


El índice de 'Batman' es 1


# Distancia entre textos

[Jellyfish](https://jellyfish.readthedocs.io/en/latest/) es una librería que contiene funciones para el cálculo de similitud entre textos. Dicha similitud puede ser á nivel léxico-gráfico (strings) o fonético.


Algoritmos de comparación de strings:

- Levenshtein Distance
- Damerau-Levenshtein Distance
- Jaro Distance
- Jaro-Winkler Distance
- Match Rating Approach Comparison
- Hamming Distance

Algoritmos de encoding fonético:

- American Soundex
- Metaphone
- NYSIIS (New York State Identification and Intelligence System)
- Match Rating Codex


In [30]:
# !pip3 install jellyfish
!pip install jellyfish

import jellyfish



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 6.2 MB/s eta 0:00:00


https://pypi.org/project/jellyfish/

## Levenshtein

Recordemos: distancia de Edit (Edición) en la que las operaciones permitidas son la inserción, la eliminación y la sustitución.

In [31]:
jellyfish.levenshtein_distance('Cisne negro', 'Cisne negro')

0

In [32]:
jellyfish.levenshtein_distance('Cisne negro', 'Cisne negor')

2

In [33]:
jellyfish.levenshtein_distance('Cisne negro', 'Cisne nego')

1

In [34]:
jellyfish.levenshtein_distance('Cisnee negro', 'Cisne nego')

2

In [35]:
jellyfish.levenshtein_distance('Cisneee negro', 'Cisne nego')

3

## Damerau-Levenshtein

Recordemos: distancia de Edit (Edición) en la que las operaciones permitidas son la inserción, la eliminación y la transposición de 2 caracteres adyacentes.

In [36]:
jellyfish.damerau_levenshtein_distance('Cisne negro', 'Cisne negro')

0

In [37]:
jellyfish.damerau_levenshtein_distance('Cisne negro', 'Cisne negor')

1

In [38]:
jellyfish.damerau_levenshtein_distance('Cisne negro', 'Cisne nego')

1

In [39]:
jellyfish.damerau_levenshtein_distance('Cisnee negro', 'Cisne nego')

2

In [41]:
jellyfish.damerau_levenshtein_distance('cisneee negro', 'Cisne nego')

4